In [2]:
import numpy as np
import pandas as pd
import nltk
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import re
import random 
import collections
# use multiprocessing for multicore ops
import copy

In [3]:
data2 = pd.read_csv("../data/questions.csv", delimiter=',', encoding="utf-8-sig")
data1 = copy.deepcopy(data2[:50000])
print(data1.shape)
print(data1.iloc[0:1,3:])
#random.shuffle(data1)
data1.dropna(inplace=True)

(50000, 6)
                                           question1  \
0  What is the step by step guide to invest in sh...   

                                           question2  is_duplicate  
0  What is the step by step guide to invest in sh...             0  


In [5]:
X = data1[["question1","question2"]]
y = data1[['is_duplicate']]


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [7]:
#nltk.download()
from nltk.corpus import stopwords # Import the stop word list
print(stopwords.words("english")) 


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [8]:
def question_to_words( raw_quest, lower=True, remove_not = False ):
    
    lemmetizer = nltk.WordNetLemmatizer()
    letters_only = re.sub("[^a-zA-Z]", " ", raw_quest)
    if lower==True:
        letters_only = letters_only.lower()
    words = nltk.word_tokenize(letters_only)
    #words = nltk.word_tokenize(s)
    stops = set(stopwords.words("english")) 
    not_stop = ['no', 'nor', 'not', "don't", "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
    if remove_not == True:
        #stop = stops.difference(not_stop)
        stops = ['the','a','an','and','but','if','or','because','as','what','which','this','that','these','those','then',
              'just','so','than','such','both','through','about','for','is','of','while','during','to','What','Which',
              'Is','If','While','This']
    meaningful_words = [w for w in words if not w in stops]
    
    meaningful_words = [lemmetizer.lemmatize(w) for w in meaningful_words]
    return( " ".join( meaningful_words ))

In [9]:
#X_train['question2']= X_train['question2'].apply(question_to_words)
#X_train = X_train.apply(question_to_words)
a = X_train['question1']
a = a.apply(question_to_words)
a
b = X_train['question2']
b = a.apply(question_to_words)



In [29]:
c=a+b

In [10]:
print("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer

Creating the bag of words...



In [11]:
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 
#add whitelisting
# use ngrams , ngram_range=(1,2) check for various combinations

In [30]:
#corpus = X_train["question1"] + " " + X_train["question2"]
corpus = a+b
cv = vectorizer.fit(corpus)

cv
from sklearn.metrics.pairwise import cosine_similarity
# #subset = corpus + y_tra
# y_train[y_train.is_duplicate==0]

In [14]:
train_dup = subset.loc[subset.is_duplicate==1,['question1','question2']]
train_nondupes = subset.loc[subset.is_duplicate==0,['question1','question2']]

NameError: name 'subset' is not defined

In [ ]:
subset